# Society-based Cooperation - EVAC Assessment 2

## Agent Representation

To begin with, each agent needs to be able to encode and store an evolvable strategy.

When defining an agents memory, there are $n^{2m}$ possible cases, where n is the number of societies types and m is the number of games each agent is allowed to remember. In this proposed solution, each agent will have a memory of 1 match, therefore there are $4^2=16$ possibilities.

The encoded strategy is simply a rule which specifies the action the agent should take in each of these 16 cases. In the context of this problem, the action to take is defined as the society the agent should switch to.

 An example subsection of this strategy is shown below --> 

| Memory              | Strategy (Society to switch to)       |
| ------------------  | ------------------------------------  |
| (Saint, Saint)      | Buddy                                 |
| (Saint, Buddy)      | Vandal                                |
| (Saint, Fight Club) | Saint                                 |
| (Saint, Vandal)     | Saint                                 |
| ...                 | ...                                   |
| (Vandal, Vandal)    | Fight Club                            |


This strategy can be expressed compactly as a list of length $4^2$ (e.g. `[Buddy, Vandal, Saint, Saint, ..., Fight Club]`). Therefore each agent must be able to find the index for the chromosome from the remembered match pairing.

Rather than generating a lookup table which maps society pairings to chromosome indexes, e.g. `{(Saint, Saint) = 0, (Saint, Buddy) = 1, ... (Vandal, Vandal) = 15}`, a faster method which also uses less memory is to map each society to a number between 0 and 3 and use the base 4 integer value of the pairing to access the chomosome. For example, with the assignments `{Saints: 0, Buddies: 1, Fight Club: 2, Vandals: 3}` the following indexes are obtained.

- (Saint, Saint) = $00_4$ maps to index 0
- (Saint, Buddy) = $01_4$ maps to index 1
- (Saint, Fight Club) = $02_4$ maps to index 2
- ...
- (Vandal, Vandal) = $33_4$ maps to index 15

**Summary**

To summarize, each agent will have a memory of 1 match, a pairing of the agents current faction and the opponents current faction. The base 4 representation of this matching will be used as the index to access the agents chromosome. This is a list of length 16 where each gene represents a society to switch to. 


## Code
### Setup
The following code imports any libraries needed to run the genetic algorithm, including the Distributed Evolutionary Algorithms in Python (DEAP) library which is an evolutionary computation framework allowing for rapid prototyping and testing of ideas. 

NumPy and Matplotlib are used for statistical analysis and visualisation of the genetic algorithms performance.

A seed in the random module has been set to ensure the submitted PDF matches the provided colab notebook.

An Enum for the societies has also been defined so they can be easily referenced throughout the program. Each one is assigned a string value between 0 to 3.

In [ ]:
import random
import itertools
import time

from enums import Society
from deap import creator, base, tools
from collections import Counter

from enum import Enum

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(100)

class Society(Enum):
    """ Enum which represents all possible societies that agents can join"""
    SAINTS = "0"
    BUDDIES = "1"
    FIGHT_CLUB = "2"
    VANDALS = "3"

## The Game
The following code is used for the game logic and determining the outcome of the rounds 

In [ ]:
# Dictionary which maps society pairings to their respective playoffs. 
OUTCOMES = {
    # Saints cooperate with everyone
    (Society.SAINTS, Society.SAINTS):           (4,4), # Both cooperate
    (Society.SAINTS, Society.BUDDIES):          (0,6), # Buddies are selfish
    (Society.SAINTS, Society.FIGHT_CLUB):       (4,4), # Both cooperate
    (Society.SAINTS, Society.VANDALS):          (0,6), # Vandals are selfish

    # Buddies only cooperate with each other
    (Society.BUDDIES, Society.SAINTS):          (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.BUDDIES):         (4,4), # Both cooperate
    (Society.BUDDIES, Society.FIGHT_CLUB):      (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.VANDALS):         (1,1), # Both selfish
    
    # Fight club cooperate with everyone but themselves
    (Society.FIGHT_CLUB, Society.SAINTS):       (4,4), # Both cooperate
    (Society.FIGHT_CLUB, Society.BUDDIES):      (0,6), # Buddies are selfish
    (Society.FIGHT_CLUB, Society.FIGHT_CLUB):   (1,1), # Both selfish
    (Society.FIGHT_CLUB, Society.VANDALS):      (0,6), # Vandals are selfish
    
    # Vandals cooperate with no one
    (Society.VANDALS, Society.SAINTS):          (6,0), # Vandals are selfish
    (Society.VANDALS, Society.BUDDIES):         (1,1), # Both selfish
    (Society.VANDALS, Society.FIGHT_CLUB):      (6,0), # Vandals are selfish
    (Society.VANDALS, Society.VANDALS):         (1,1), # Both selfish
}

def play_round(indiv1, indiv2):
    '''Updates each individual depending on the outcome of the round played'''
    # Increments each individuals rounds_played attribute
    indiv1.rounds_played += 1
    indiv2.rounds_played += 1
    
    # Increases each individuals wealth by the appropriate payoff
    payoffs = OUTCOMES[(indiv1.society, indiv2.society)]
    indiv1.total_score += payoffs[0]
    indiv2.total_score += payoffs[1]

    # Queries each agents chromosome depending on the match played to decided whether or not to switch society.
    new_match = str(indiv1.society.value)+(indiv2.society.value)

    chr1_index = int(new_match, base=4)
    indiv1.society = Society(indiv1[chr1_index])

    chr2_index = int(new_match[::-1], base=4)
    indiv2.society = Society(indiv2[chr2_index])

In [ ]:



def reset_population(population):
    for indiv in population:
        indiv.total_score = 0
        indiv.rounds_played = 0
        indiv.society = random.choice(list(Society))
        indiv.history = "".join([random.choice([society.value for society in Society]) for i in range(2)])

def evaluate_agents(population, num_rounds):
    for i in range(num_rounds):
        indiv1 = random.choice(population)
        indiv2 = random.choice(population)
        while indiv2 == indiv1:
            indiv2 = random.choice(population)
        play_round(indiv1, indiv2)
    
    for ind in population:
        if ind.rounds_played == 0:
            ind.fitness.values = 0,
        else:
            ind.fitness.values = ind.total_score / ind.rounds_played,

IND_SIZE = 4**2

def run_GA(gen_num=200, pop_num=20, round_num=400, mut_prob=0.021, cx_prob=0.15, tourn_size=5, headless=True):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax, total_score=0, rounds_played=0, society=Society.SAINTS, history = "00")

    toolbox = base.Toolbox()
    toolbox.register("attr_int", random.choice, [society.value for society in Society])
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n = IND_SIZE)

    toolbox.register("evaluate", evaluate_agents)

    # Registers function to select, mate and mutate individuals
    toolbox.register("select", tools.selTournament, tournsize=tourn_size)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mut_prob)

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("reset_population", reset_population)

    # Registers the statistics & logbook that will be logged during the GA
    stats = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats2 = tools.Statistics(key=lambda ind: ind.society.value)

    stats.register("mean", np.mean)
    stats.register("std", np.std)
    stats.register("median", np.median)
    stats.register("min", np.min)
    stats.register("max", np.max)

    logbook = tools.Logbook()
    logbook2 = tools.Logbook()

    population = toolbox.population(pop_num)
    toolbox.reset_population(population)

    toolbox.evaluate(population, round_num)

    counts = {
        Society.SAINTS: [],
        Society.BUDDIES: [],
        Society.FIGHT_CLUB: [],
        Society.VANDALS: []
    }

    # Genetic Algorithm
    for g in range(gen_num):
        print(f"-- Running Generation {g+1}/{gen_num} --       ", end='\r')

        # Selects number of individuals equal to population length
        offspring = toolbox.select(population, len(population))
        
        # Includes duplicates so clones all individuals
        offspring = list(map(toolbox.clone, offspring))

        # Performs crossover on 2 individuals based on previously defined probability
        for indiv1, indiv2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_prob:
                toolbox.mate(indiv1, indiv2)

        # Mutates offspring based on previously defined probability #TODO: Modify probability/algorithm type?
        for mutant in offspring:
            toolbox.mutate(mutant)
        
        toolbox.reset_population(offspring)

        # Recalculates fitness values for mutated offspring
        toolbox.evaluate(offspring, round_num)

        # Replaces old population with new mutated offspring
        population[:] = offspring

        # Compiles & records the statistics for the new generation
        
        record = stats.compile(population)
        logging.debug(record)
        logbook.record(gen=g, **record)

        assignments = [indiv.society for indiv in population]
        for society in Society:
            counts[society].append(assignments.count(society))

    return logbook, counts


In [ ]:
lb, counts = run_GA(gen_num=200, pop_num=2000, round_num=10000, mut_prob=0.05, cx_prob=0.15, tourn_size=200, headless=True)

In [ ]:
def draw_graphs(logbook, counts):
    gens = logbook.select("gen")
    means = logbook.select("mean")

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(14, 8)
    fig.suptitle("Society Based Cooperation")

    ax1.set_title("Fitness Stats")
    ax1.plot(gens, means, lw=3, color="red")

    ax2.set_title("Assignment Counts")
    ax2.plot(gens, counts[Society.SAINTS], lw=2, color="green", label="Saints")
    ax2.plot(gens, counts[Society.BUDDIES], lw=2, color="lightblue", label="Buddies")
    ax2.plot(gens, counts[Society.FIGHT_CLUB], lw=2, color="orange", label="Fight Club")
    ax2.plot(gens, counts[Society.VANDALS], lw=2, color="red", label="Vandals")
    ax2.legend

    plt.show()

In [ ]:
draw_graphs(lb, counts)